In [154]:
from pyspark.sql import HiveContext
from pyspark import SparkContext
from pyspark.sql.functions import *   #importar todas as funções do pyspark sql
from pyspark.sql.types import *       #importar as conversoes
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

In [2]:
sqlContext = HiveContext(sc)

In [3]:
sqlContext.sql("show databases").show()

+------------+
|databaseName|
+------------+
|       covid|
|     default|
+------------+



In [50]:
sqlContext.sql("use covid")

DataFrame[]

In [51]:
sqlContext.sql("show tables").show()

+--------+-----------------+-----------+
|database|        tableName|isTemporary|
+--------+-----------------+-----------+
|   covid|            covid|      false|
|   covid|covid_partitioned|      false|
+--------+-----------------+-----------+



In [53]:
covid = sqlContext.sql('select * from covid_partitioned')

In [58]:
covidDF = spark.read.table('covid_partitioned')
covidDF.printSchema()

root
 |-- regiao: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- coduf: integer (nullable = true)
 |-- codmun: integer (nullable = true)
 |-- codregiaosaude: integer (nullable = true)
 |-- nomeregiaosaude: string (nullable = true)
 |-- data: string (nullable = true)
 |-- semanaepi: integer (nullable = true)
 |-- populacaotcu219: integer (nullable = true)
 |-- casosacumulados: integer (nullable = true)
 |-- casosnovos: integer (nullable = true)
 |-- obitosacumulados: integer (nullable = true)
 |-- obitosnovos: integer (nullable = true)
 |-- recuperadosnovos: integer (nullable = true)
 |-- emacompanhamentonovos: integer (nullable = true)
 |-- interiormetropolitana: integer (nullable = true)
 |-- municipio: string (nullable = true)



In [97]:
covidFirstDF = covidDF.agg(sum("recuperadosnovos").alias("Emacompanhamento"), max("emacompanhamentonovos").alias("CasosRecuperados"))
covidFirstDF.show()

+----------------+----------------+
|Emacompanhamento|CasosRecuperados|
+----------------+----------------+
|         1580676|        17262646|
+----------------+----------------+



In [98]:
covidFirstDF.write.saveAsTable("covidFirstView")

In [144]:
covidSecDF = covidDF.agg(max("casosnovos").alias("Acumulado"), last("casosacumulados").alias("CasosNovos"))
covidSecDF.show()

+---------+----------+
|Acumulado|CasosNovos|
+---------+----------+
| 18855015|   3015268|
+---------+----------+



In [145]:
covidSecDF.write.parquet('/user/marcel/data/covid/covidSecView_teste', compression="snappy")

In [146]:
covidThDF= covidDF.agg(max("obitosnovos").alias("ObitosAcumulados"))
covidThDF.show()

+----------------+
|ObitosAcumulados|
+----------------+
|          526892|
+----------------+



In [163]:
#covidThDF.write.json('/user/marcel/data/covid/covidThView')
covidThDF.select(from_json(col("ObitosAcumulados").cast("string"), schema="schema")).writeStream

ParseException: "\nmismatched input '<EOF>' expecting {'SELECT', 'FROM', 'ADD', 'AS', 'ALL', 'ANY', 'DISTINCT', 'WHERE', 'GROUP', 'BY', 'GROUPING', 'SETS', 'CUBE', 'ROLLUP', 'ORDER', 'HAVING', 'LIMIT', 'AT', 'OR', 'AND', 'IN', NOT, 'NO', 'EXISTS', 'BETWEEN', 'LIKE', RLIKE, 'IS', 'NULL', 'TRUE', 'FALSE', 'NULLS', 'ASC', 'DESC', 'FOR', 'INTERVAL', 'CASE', 'WHEN', 'THEN', 'ELSE', 'END', 'JOIN', 'CROSS', 'OUTER', 'INNER', 'LEFT', 'SEMI', 'RIGHT', 'FULL', 'NATURAL', 'ON', 'PIVOT', 'LATERAL', 'WINDOW', 'OVER', 'PARTITION', 'RANGE', 'ROWS', 'UNBOUNDED', 'PRECEDING', 'FOLLOWING', 'CURRENT', 'FIRST', 'AFTER', 'LAST', 'ROW', 'WITH', 'VALUES', 'CREATE', 'TABLE', 'DIRECTORY', 'VIEW', 'REPLACE', 'INSERT', 'DELETE', 'INTO', 'DESCRIBE', 'EXPLAIN', 'FORMAT', 'LOGICAL', 'CODEGEN', 'COST', 'CAST', 'SHOW', 'TABLES', 'COLUMNS', 'COLUMN', 'USE', 'PARTITIONS', 'FUNCTIONS', 'DROP', 'UNION', 'EXCEPT', 'MINUS', 'INTERSECT', 'TO', 'TABLESAMPLE', 'STRATIFY', 'ALTER', 'RENAME', 'ARRAY', 'MAP', 'STRUCT', 'COMMENT'